<a href="https://colab.research.google.com/github/mafaldasalomao/Transfering_learning_TF-ExtractFeatures/blob/main/04_transfering_learning_TF_partI_Feature_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
#download  the data
# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

#unzip the downloaded data
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()


--2022-10-07 19:27:37--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 142.250.4.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   346MB/s    in 0.5s    

2022-10-07 19:27:38 (346 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
#how many images in each flder?
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories in {len(filenames)} in the {dirpath}.")

There are 2 directories in 0 in the 10_food_classes_10_percent.
There are 10 directories in 0 in the 10_food_classes_10_percent/test.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/chicken_wings.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/steak.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/hamburger.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/fried_rice.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/ice_cream.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/chicken_curry.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/ramen.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/sushi.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/grilled_salmon.
There are 0 directories in 250 in the 10_food_classes_10_percent/test/pizza.
There are 10 directories in 0 in the 10_food_classes_10_pe

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode="categorical")


Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [4]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_call = tf.keras.callbacks.tensorboard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback